# 1 Create Vector Space Model

In [13]:
import pandas as pd
import re
import string
from nltk.corpus import stopwords
import nltk
import spacy
from tqdm import tqdm 
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
stemmer = PorterStemmer()
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\halil\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [14]:
df = pd.read_csv("SmallDataset/PreprocessedContents.csv")

In [15]:
df.head()

,DocumentName,Content,PreprocessedContent,ContentLength
0,output_1.txt,Science & Mathematics PhysicsThe hot glowing s...,scienc mathemat physic hot glow surfac star em...,889
1,output_10.txt,"""Health Other - HealthI have trouble swallowin...",health health troubl swallow due ms crush vali...,1012
2,output_100.txt,"""Supertramp The Logical Song without Cofounder...",supertramp logic song without cofound roger ho...,1705
3,output_1000.txt,"""Legal market overview‘ France is shrinking! ’...",legal market overview franc shrink lament one ...,6691
4,output_1001.txt,"""Name ArgonSymbol ArAtomic Number 18Atomic Mas...",name argon symbol ar atom number atom mass ato...,862


In [16]:
contents = df[["DocumentName","PreprocessedContent"]]
contents['DocumentName'] = contents['DocumentName'].str.replace('output_', '').str.replace('.txt', '')

C:\Users\halil\AppData\Local\Temp\ipykernel_13336\2463016382.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  contents['DocumentName'] = contents['DocumentName'].str.replace('output_', '').str.replace('.txt', '')


In [17]:
contents.head()

,DocumentName,PreprocessedContent
0,1,scienc mathemat physic hot glow surfac star em...
1,10,health health troubl swallow due ms crush vali...
2,100,supertramp logic song without cofound roger ho...
3,1000,legal market overview franc shrink lament one ...
4,1001,name argon symbol ar atom number atom mass ato...


In [18]:
class TextPreprocessor:
    def __init__(self):
        self.stop_words = set(stopwords.words('english'))
        self.url_pattern = re.compile(r'https?://\S+')

    def split_words(self, text):
        return re.sub(r'([a-z])([A-Z])', r'\1 \2', text)

    def to_lowercase(self, text):
        return text.lower() if isinstance(text, str) else text

    def remove_urls(self, text):
        return self.url_pattern.sub('', text)

    def remove_nonword_chars(self, text):
        text = re.sub(r'[^\w\s]', ' ', text)
        return re.sub(r'[^a-zA-Z\s]', ' ', text)

    def remove_newlines(self, text):
        return text.replace('\n', ' ')

    def remove_digits(self, text):
        return re.sub(r'\d', ' ', text)

    def remove_stopwords(self, text):
        words = text.split()
        filtered_words = [word for word in words if word.lower() not in self.stop_words]
        return ' '.join(filtered_words)

    def remove_punctuation(self, text):
        return ''.join([' ' if char in string.punctuation else char for char in text])

    def remove_one_letter_words(self, text):
        return ' '.join([word for word in text.split() if len(word) > 1])
    
    def remove_multiple_spaces(self, text):
        return re.sub(r'\s+', ' ', text)

    def strip_text(self, text):
        return text.strip()
    
    def stem_words(self,text):
        word_tokens = text.split()
        stems = ' '.join([stemmer.stem(word) for word in word_tokens])
        return stems

    def preprocess(self, text):
        text = self.split_words(text)
        text = self.to_lowercase(text)
        text = self.remove_urls(text)
        text = self.remove_nonword_chars(text)
        text = self.remove_newlines(text)
        text = self.remove_digits(text)
        text = self.remove_stopwords(text)
        text = self.remove_punctuation(text)
        text = self.remove_one_letter_words(text)
        text = self.remove_multiple_spaces(text)
        text = self.strip_text(text)
        text = self.stem_words(text)
        return text

In [25]:
preprocessor = TextPreprocessor()

In [27]:
#tfidf = TfidfVectorizer(ngram_range=(1,3),min_df=0.05)
#tfidf = TfidfVectorizer(max_df=0.95, min_df=2, max_features=10000)
tfidf = TfidfVectorizer(ngram_range=(1,3))

In [29]:
tfidf_matrix = tfidf.fit_transform(contents['PreprocessedContent'])

In [30]:
terms = tfidf.get_feature_names_out()

In [31]:
terms.shape

(1607227,)

### Example Document Retrieval

In [602]:
query = "what agency can i report a scammer concerning my computer"

preprocessed_query = preprocessor.preprocess(query)

In [603]:
preprocessed_query

'agenc report scammer concern comput'

In [604]:
query_vector = tfidf.transform([preprocessed_query])

In [605]:
cosine_similarities = cosine_similarity(query_vector, tfidf_matrix)

In [606]:
cosine_similarities

array([[0.        , 0.00697789, 0.        , ..., 0.        , 0.00723635,
        0.        ]])

In [607]:
similarity_df = pd.DataFrame(cosine_similarities.T, index=contents['DocumentName'], columns=['CosineSimilarity'])

In [608]:
ranked_docs = similarity_df.sort_values(by='CosineSimilarity', ascending=False)

In [609]:
ranked_docs.head(50)

,CosineSimilarity
DocumentName,
1478,0.141620
915,0.072077
697,0.053506
1511,0.052207
1122,0.050464
1206,0.049622
1296,0.047018
1175,0.040748
1301,0.027098


### Testing VSM

In [35]:
df_dev_query_results_small = pd.read_csv("SmallDataset/dev_query_results_small.csv")
df_dev_small_queries = pd.read_excel("SmallDataset/dev_small_queries.xlsx")

In [36]:
df_dev_small_queries.head()

,Query number,Query
0,1089273,types of road hugger tires
1,1087869,what agency can i report a scammer concerning ...
2,1087858,what airport is in wilder ky
3,1087532,what are isotopes quizlet
4,1086886,what are the products and by products of photo...


In [39]:
df_merged_results = df_dev_query_results_small.merge(df_dev_small_queries,left_on="Query_number",right_on="Query number",)

In [41]:
df_merged_results = df_merged_results.rename(columns={"doc_number":"DocumentName"})

In [43]:
df_merged_results.head()

,Query_number,DocumentName,Query number,Query
0,1089273,590,1089273,types of road hugger tires
1,1087869,1478,1087869,what agency can i report a scammer concerning ...
2,1087858,547,1087858,what airport is in wilder ky
3,1087532,595,1087532,what are isotopes quizlet
4,1086886,466,1086886,what are the products and by products of photo...


### Compute MAP@K

In [46]:
contents.head()

,DocumentName,PreprocessedContent
0,1,scienc mathemat physic hot glow surfac star em...
1,10,health health troubl swallow due ms crush vali...
2,100,supertramp logic song without cofound roger ho...
3,1000,legal market overview franc shrink lament one ...
4,1001,name argon symbol ar atom number atom mass ato...


In [48]:
def calculate_map_at_k(K,df_merged_results=df_merged_results,contents=contents):
    average_precisions = []
    
    for index, row in df_merged_results.iterrows():
        query_text = row['Query']
        actual_document = row['DocumentName']
        preprocessed_query = preprocessor.preprocess(query_text)
        query_vector = tfidf.transform([preprocessed_query])
        cosine_similarities = cosine_similarity(query_vector, tfidf_matrix)#.flatten()
        similarity_df = pd.DataFrame(cosine_similarities.T, index=contents['DocumentName'], columns=['CosineSimilarity'])
        ranked_docs = similarity_df.sort_values(by='CosineSimilarity', ascending=False)
        top_k_docs = ranked_docs.head(K).index.astype(int).tolist()

        relevant_count = 0
        precision_at_k = []
        for i, doc_name in enumerate(top_k_docs, start=1):
            if doc_name == actual_document:
                relevant_count += 1
                precision_at_k.append(relevant_count / i)

        if precision_at_k:
            avg_precision = sum(precision_at_k) / min(K, len(precision_at_k))
        else:
            avg_precision = 0

        average_precisions.append(avg_precision)
        
    map_at_k = sum(average_precisions) / len(average_precisions)
    return round(map_at_k, 4)

#### Calculate MAP@K=3

In [51]:
calculate_map_at_k(3)

0.7207

#### Calculate MAP@K=10

In [54]:
calculate_map_at_k(10)

0.7449

### Compute MAR@K

In [64]:
def mean_average_recall_at_k(top_k_docs, relevant_docs):
    num_relevant_retrieved = len(set(top_k_docs.index) & set(relevant_docs))
    num_relevant_total = len(relevant_docs)

    if num_relevant_total > 0:
        return num_relevant_retrieved / num_relevant_total
    else:
        return 0.0

#### Calculate MAR@K=3

In [66]:
unique_queries = df_merged_results['Query'].unique()
k = 3

In [70]:
results = []
counter = 0 
test_results_df_top3 = pd.DataFrame(columns = ["Query_number","doc_number"])
for query in unique_queries:
    print(counter)
    counter+=1
    preprocessed_query = preprocessor.preprocess(query)  
    query_vector = tfidf.transform([preprocessed_query])
    cosine_similarities = cosine_similarity(query_vector, tfidf_matrix)
    similarity_df = pd.DataFrame(cosine_similarities.T, index=contents['DocumentName'], columns=['CosineSimilarity'])
    top_k_docs = similarity_df.sort_values(by='CosineSimilarity', ascending=False).head(k)
    relevant_docs = df_merged_results[df_merged_results['Query'] == query]['DocumentName'].astype(str).tolist()
    query_number = str(df_merged_results[df_merged_results['Query'] == query]["Query_number"].values[0])
    new_records = pd.DataFrame({
    'Query_number': [query_number] * 3,  # Repeat 155 for each new row
    'doc_number': top_k_docs.index  # Values for B
    })
    test_results_df_top3 = pd.concat([test_results_df_top3, new_records], ignore_index=True)
    mar_k = mean_average_recall_at_k(top_k_docs, relevant_docs)
    results.append({'Query': query, 'MAR@K': mar_k})

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247


In [88]:
sum_mar3 = 0
for i in results:
    sum_mar3 += i["MAR@K"]
mar3 = sum_mar3 / len(results)

In [90]:
mar3

0.8125

#### Calculate MAR@K=10

In [92]:
unique_queries = df_merged_results['Query'].unique()
k = 10

In [94]:
results_top10 = []
counter = 0 
test_results_df_top10 = pd.DataFrame(columns = ["Query_number","doc_number"])
for query in unique_queries:
    counter+=1
    preprocessed_query = preprocessor.preprocess(query)  
    query_vector = tfidf.transform([preprocessed_query])
    cosine_similarities = cosine_similarity(query_vector, tfidf_matrix)
    similarity_df = pd.DataFrame(cosine_similarities.T, index=contents['DocumentName'], columns=['CosineSimilarity'])
    top_k_docs = similarity_df.sort_values(by='CosineSimilarity', ascending=False).head(k)
    relevant_docs = df_merged_results[df_merged_results['Query'] == query]['DocumentName'].astype(str).tolist()
    query_number = str(df_merged_results[df_merged_results['Query'] == query]["Query_number"].values[0])
    new_records = pd.DataFrame({
    'Query_number': [query_number] * k,  # Repeat 155 for each new row
    'doc_number': top_k_docs.index  # Values for B
    })
    test_results_df_top10 = pd.concat([test_results_df_top10, new_records], ignore_index=True)
    mar_k = mean_average_recall_at_k(top_k_docs, relevant_docs)
    results_top10.append({'Query': query, 'MAR@K': mar_k})

In [96]:
sum_mar10 = 0
for i in results_top10:
    sum_mar10 += i["MAR@K"]
mar10 = sum_mar10 / len(results)

In [98]:
mar10

0.9294354838709677